# VQE for H2 with QSD Diagnostics

This notebook runs a small **VQE** example for the hydrogen molecule (H2) using Qiskit and then visualizes the state evolution using **QSD**.

We capture three iterations (**init**, **mid**, **final**) and provide: 
- QSD images for each iteration
- the corresponding statevectors


## 0. Setup

Requires **Qiskit >= 2.0** and `qiskit-algorithms` installed.
This assumes Qiskit is installed. QSD is loaded from the local repo.


In [12]:
import numpy as np
from pathlib import Path

from qsd import plot_qsd

# Qiskit imports (>= 2.0)
import qiskit
from qiskit.circuit.library import TwoLocal
from qiskit.quantum_info import Statevector
from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives import StatevectorEstimator as Estimator
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SLSQP


## 1. H2 Hamiltonian (2-qubit reduction)

We use the standard 2-qubit H2 Hamiltonian at bond length 0.735 Å (as used in many Qiskit VQE tutorials).


In [13]:
# Hamiltonian coefficients for H2 (2-qubit parity mapping, frozen core)
h2_op = SparsePauliOp.from_list(
    [
        ("II", -1.052373245772859),
        ("ZI", 0.39793742484318045),
        ("IZ", -0.39793742484318045),
        ("ZZ", -0.01128010425623538),
        ("XX", 0.18093119978423156),
    ]
)

# Variational ansatz
ansatz = TwoLocal(2, ["ry", "rz"], "cx", reps=2)


/var/folders/mv/jppb68jj4495fl3hmy9jwqs40000gn/T/ipykernel_48385/3042272550.py:13: DeprecationWarning: The class ``qiskit.circuit.library.n_local.two_local.TwoLocal`` is deprecated as of Qiskit 2.1. It will be removed in Qiskit 3.0. Use the function qiskit.circuit.library.n_local instead.
  ansatz = TwoLocal(2, ["ry", "rz"], "cx", reps=2)


## 2. Run VQE and collect iteration snapshots

We store parameters and energies from the VQE callback.


In [14]:
history = {"iters": [], "params": [], "energies": []}

def _flatten_params(params, expected):
    if isinstance(params, dict):
        return np.array([params[p] for p in expected], dtype=float)
    arr = np.array(params, dtype=object)
    if arr.size == 1 and isinstance(arr.item(), (list, tuple, np.ndarray)):
        arr = np.array(arr.item(), dtype=float)
    else:
        arr = np.array(params, dtype=float)
    return np.array(arr, dtype=float).ravel()

def store_callback(eval_count, params, energy, metadata):
    history["iters"].append(eval_count)
    flat = _flatten_params(params, ansatz.parameters)
    history["params"].append(flat)
    history["energies"].append(float(energy))

optimizer = SLSQP(maxiter=60)
estimator = Estimator()

vqe = VQE(estimator, ansatz, optimizer=optimizer, callback=store_callback)
result = vqe.compute_minimum_eigenvalue(h2_op)

result


## 3. Extract init / mid / final statevectors

We pick the first, middle, and last VQE iterations and compute the corresponding statevectors.


In [15]:
if not history["params"]:
    raise RuntimeError("VQE did not record any iterations.")

idx_init = 0
idx_mid = len(history["params"]) // 2
idx_final = len(history["params"]) - 1

def statevector_from_params(params):
    flat = _flatten_params(params, ansatz.parameters)
    if flat.size != ansatz.num_parameters:
        raise ValueError(
            f"Expected {ansatz.num_parameters} params, got {flat.size}."
        )
    bound = dict(zip(ansatz.parameters, flat))
    circ = ansatz.assign_parameters(bound, inplace=False)
    return Statevector.from_instruction(circ)

sv_init = statevector_from_params(history["params"][idx_init])
sv_mid = statevector_from_params(history["params"][idx_mid])
sv_final = statevector_from_params(history["params"][idx_final])

statevectors = {
    "init": sv_init.data,
    "mid": sv_mid.data,
    "final": sv_final.data,
}

statevectors


ValueError: Expected 12 params, got 1.

In [ ]:
import json
statevec_path = Path("figures/h2_vqe_statevectors.json")
statevec_path.parent.mkdir(parents=True, exist_ok=True)


serializable = {
    k: [[float(z.real), float(z.imag)] for z in v]
    for k, v in statevectors.items()
}

statevec_path.write_text(json.dumps(serializable, indent=2))
statevec_path


## 4. QSD visualization (init / mid / final)

We plot QSDs for the selected iterations and save them in `figures/`.


In [ ]:
fig_dir = Path("figures")
fig_dir.mkdir(parents=True, exist_ok=True)

plot_qsd(
    sv_init.data, 2],
    grouping="hamming",
    ordering="lex",
    theme="dark",
    save_path=str(fig_dir / "h2_vqe_qsd_init.png"),
    caption="H2 VQE | init",
)

plot_qsd(
    sv_mid.data, 2],
    grouping="hamming",
    ordering="lex",
    theme="dark",
    save_path=str(fig_dir / "h2_vqe_qsd_mid.png"),
    caption="H2 VQE | mid",
)

plot_qsd(
    sv_final.data, 2],
    grouping="hamming",
    ordering="lex",
    theme="dark",
    save_path=str(fig_dir / "h2_vqe_qsd_final.png"),
    caption="H2 VQE | final",
)


## 5. Saved artifacts

- `figures/h2_vqe_qsd_init.png`
- `figures/h2_vqe_qsd_mid.png`
- `figures/h2_vqe_qsd_final.png`
